In [ ]:

%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.event_detection import detect_rsw_and_rcap

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import get_stops_utils, collect_stop_data

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_fit import stop_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_decoding_tools import glm_decoding_llr

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import assemble_stop_design
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import raster_plot, raster_plot_3d

# import decoding
from neural_data_analysis.neural_analysis_tools.decoding_tools import decoding_utils, decoding_analysis, plot_decoding, cmp_decode, load_results
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_neural_viz import stop_event_raster

from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data

from neural_data_analysis.neural_analysis_tools.lfads import lfads_visualizer

from neural_data_analysis.neural_analysis_tools.neural_axes import window_search, axis_utils

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats
import json

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_colwidth', 200)

# retrieve data

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0326"

pn, datasets, comparisons = collect_stop_data.collect_stop_data_func(
    raw_data_folder_path)

globals().update(datasets)

captures_df, valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = get_stops_utils.prepare_no_capture_and_captures(
    monkey_information=pn.monkey_information,
    closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
    ff_caught_T_new=pn.ff_caught_T_new,
    distance_col="distance_from_ff_to_stop",
)

keys = ['rsw_first_vs_rcap_first', 'rsw_middle_vs_rcap_middle', 'one_stop_vs_both_first_miss', 'switch_vs_retry_after_miss', 'switch_vs_retry_after_retry']

cfg = core_stops_psth.PSTHConfig(
    pre_window=0.5,
    post_window=0.5,
    bin_width=0.05,
    smoothing_sigma=0.1,
    min_trials=5,
    normalize="zscore",            # try: None, "sub", or "div"
)

# get stop_label_df

In [ ]:
cols = ['stop_id', 'time', 'point_index', 'stop_id_start_time', 'stop_id_end_time', 'stop_id_duration']
stop_label_df = pn.stop_category_df[cols].copy()


In [ ]:
combd_retry_df = pd.concat([datasets['rcap'], datasets['rsw']], axis=0).reset_index(drop=True)
datasets['combd_retry_first'] = combd_retry_df[combd_retry_df['is_first']].copy()
datasets['combd_retry_second'] = combd_retry_df[combd_retry_df['is_second']].copy()
datasets['combd_retry_third'] = combd_retry_df[combd_retry_df['is_third']].copy()
datasets['combd_retry_last'] = combd_retry_df[combd_retry_df['is_last']].copy()
datasets['rcap_second_to_last'] = combd_retry_df[combd_retry_df['is_second_to_last']].copy()
datasets['rcap_before_second_to_last'] = combd_retry_df[combd_retry_df['before_second_to_last']].copy()

In [ ]:

for col_name, df, value in [
    ('capture', datasets['captures'], 1),
    ('as_retry', datasets['retry_after_miss'], 1),
    ('will_retry', pd.concat([datasets['rcap_nonfinal'], datasets['rsw_nonfinal']], axis=0), 1),
    ('first_retry', datasets['combd_retry_first'], 1),
    ('second_retry', datasets['combd_retry_second'], 1),
    ('third_retry', datasets['combd_retry_third'], 1),
    ('last_retry', datasets['combd_retry_last'], 1),
    ('attempt', datasets['non_captures_minus_all_misses'], 0),
    ('next_retry_capture', datasets['rcap_second_to_last'], 1),
    ('next_retry_miss', pd.concat([datasets['rsw_nonfinal'], datasets['rcap_before_second_to_last']], axis=0), 1),
]:
    
    df = df.copy()
    df[col_name] = value
    stop_label_df.drop(columns=[col_name], inplace=True, errors='ignore')
    stop_label_df = pd.merge(stop_label_df, df[['stop_id', col_name]], on='stop_id', how='left')
    stop_label_df[col_name] = stop_label_df[col_name].fillna(1-value)

# retry x geometry interaction

In [ ]:
columns = ['capture', 'as_retry', 'will_retry', 'first_retry', 'second_retry', 'third_retry', 'last_retry', 'attempt', 'next_retry_capture', 'next_retry_miss']

In [ ]:
stop_label_df = stop_label_df[stop_label_df['capture']==0].copy().drop(columns=['capture'])
columns.remove('capture')

In [ ]:
from neural_data_analysis.neural_analysis_tools.neural_axes import multivariate_axis_analyzer

analyzer = multivariate_axis_analyzer.MultivariateAxisAnalyzer(
    spikes_df=pn.spikes_df,
    behavior_df=stop_label_df,
    event_col='time',
    behavior_cols=columns,
    bin_width_ms=10,
)

axis_info = analyzer.compute_axes(
    window_ms=(100, 200),
    interaction_pairs=[(0, 1)],   # retry × angular_error
    rank=10,
)

W = axis_info['axes']
proj = axis_info['projection']


In [ ]:
for r in range(1, 10):
    axis_info = analyzer.compute_axes(window_ms=(100, 200), rank=r)
    print(r, axis_info['projection'].var(axis=0).sum())


# analyze results

In [ ]:
print(W.shape)          # (n_neurons, 3)
print(proj.shape)       # (T, 3)
assert proj.shape[0] == analyzer.fr_mat.shape[0]


In [ ]:
axis_var = proj.var(axis=0)

order = np.argsort(axis_var)[::-1]
W = W[:, order]
proj = proj[:, order]
axis_var = axis_var[order]

axis_var / axis_var.sum()



### Plot each axis separately

In [ ]:
event_bins = analyzer._events_to_bins(stop_label_df['time'].to_numpy())

window_ms = (-300, 500)
so = int(window_ms[0] / analyzer.bin_width_ms)
eo = int(window_ms[1] / analyzer.bin_width_ms)
offsets = np.arange(so, eo)

T = proj.shape[0]
idx = event_bins[:, None] + offsets[None, :]
idx = np.clip(idx, 0, T - 1)

aligned_proj = proj[idx]   # (n_stops, time, 3)
time = offsets * analyzer.bin_width_s

for k in range(10):
    if k >= aligned_proj.shape[2]:
        break
    mean = aligned_proj[:, :, k].mean(axis=0)
    sem = aligned_proj[:, :, k].std(axis=0) / np.sqrt(aligned_proj.shape[0])

    plt.figure()
    plt.fill_between(time, mean - sem, mean + sem, alpha=0.3)
    plt.plot(time, mean)
    plt.axvline(0, color='k', linestyle='--')
    plt.title(f'Axis {k+1}')
    plt.xlabel('Time from stop (s)')
    plt.ylabel('Projection')
    plt.show()



## Relate axes back to behavior

In [ ]:
stop_proj = aligned_proj[:, (offsets >= 0) & (offsets <= 20), :].mean(axis=1)

for col in columns:
    y = stop_label_df[col].to_numpy()
    print(col)
    for k in range(3):
        plt.figure()
        plt.boxplot([stop_proj[y==0, k], stop_proj[y==1, k]])
        plt.xticks([1, 2], ['Not ' + col, '' + col])
        plt.title(f'Axis {k+1}')
        plt.show()


## Conditioned peri-stop plots

In [ ]:
event_bins = analyzer._events_to_bins(stop_label_df['time'].to_numpy())

window_ms = (-300, 500)
so = int(window_ms[0] / analyzer.bin_width_ms)
eo = int(window_ms[1] / analyzer.bin_width_ms)
offsets = np.arange(so, eo)

idx = event_bins[:, None] + offsets[None, :]
idx = np.clip(idx, 0, proj.shape[0] - 1)

aligned_proj = proj[idx]            # (n_stops, n_timepoints, n_axes)
time = offsets * analyzer.bin_width_s

### Split by condition

In [ ]:
stop_label_df.columns

In [ ]:
axis_idx = 0

y = stop_label_df['first_retry'].to_numpy().astype(bool)

aligned_last = aligned_proj[y]       # last_retry == 1
aligned_not  = aligned_proj[~y]      # last_retry == 0

mean_last = aligned_last[:, :, axis_idx].mean(axis=0)
sem_last  = aligned_last[:, :, axis_idx].std(axis=0) / np.sqrt(aligned_last.shape[0])

mean_not = aligned_not[:, :, axis_idx].mean(axis=0)
sem_not  = aligned_not[:, :, axis_idx].std(axis=0) / np.sqrt(aligned_not.shape[0])


### plot

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))

plt.fill_between(time, mean_last - sem_last, mean_last + sem_last, alpha=0.3)
plt.plot(time, mean_last, label='Last retry')

plt.fill_between(time, mean_not - sem_not, mean_not + sem_not, alpha=0.3)
plt.plot(time, mean_not, label='Not last retry')

plt.axvline(0, color='k', linestyle='--')
plt.xlabel('Time from stop (s)')
plt.ylabel('Axis 1 projection')
plt.legend()
plt.title('Axis 1 conditioned on last_retry')
plt.show()


### Average Axis 1 separately

## Axis geometry

### Angles between axes

In [ ]:
from neural_data_analysis.neural_analysis_tools.neural_axes.axis_utils import axis_angle

for i in range(3):
    for j in range(i+1, 3):
        print(i, j, axis_angle(W[:, i], W[:, j]))


### Neuron loadings (who contributes?)

In [ ]:
plt.figure()
plt.imshow(W, aspect='auto', cmap='bwr')
plt.colorbar(label='Weight')
plt.xlabel('Axis')
plt.ylabel('Neuron')
plt.title('Neuron loadings on axes')
plt.show()


## Cross-validation / stability

### Angles between vectors

In [ ]:
from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=5, test_size=0.5)
Ws = []

for train, _ in rs.split(stop_label_df):
    sub_analyzer = multivariate_axis_analyzer.MultivariateAxisAnalyzer(
        spikes_df=pn.spikes_df,
        behavior_df=stop_label_df.iloc[train].reset_index(drop=True),
        event_col='time',
        behavior_cols=columns,
        bin_width_ms=10,
    )
    info = sub_analyzer.compute_axes(
        window_ms=(100, 200),
        interaction_pairs=[(0, 1)],
        rank=3,
    )
    Ws.append(info['axes'])
    
for k in range(3):
    angles = [
        axis_angle(W[:, k], W2[:, k])
        for W2 in Ws
    ]
    print(f'Axis {k+1}: mean angle = {np.mean(angles):.1f}°')



### Angles between subspaces

In [ ]:
from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=5, test_size=0.5)
Ws = []

for train, _ in rs.split(stop_label_df):
    sub_analyzer = multivariate_axis_analyzer.MultivariateAxisAnalyzer(
        spikes_df=pn.spikes_df,
        behavior_df=stop_label_df.iloc[train].reset_index(drop=True),
        event_col='time',
        behavior_cols=columns,
        bin_width_ms=10,
    )
    info = sub_analyzer.compute_axes(
        window_ms=(100, 200),
        interaction_pairs=[(0, 1)],
        rank=3,
    )
    Ws.append(info['axes'])   # each: (neurons, 3)


In [ ]:
# Pick a reference subspace
# Typically the full-data fit, or the first split:
W_ref = Ws[0]

from scipy.linalg import subspace_angles
import numpy as np

angles_all = []

for W2 in Ws[1:]:
    angles_rad = subspace_angles(W_ref, W2)   # radians, length = rank
    angles_deg = np.degrees(angles_rad)
    angles_all.append(angles_deg)

angles_all = np.array(angles_all)  # shape (n_splits-1, rank)

print('Principal angles (degrees) per split:')
print(angles_all)

print('\nMean principal angle per dimension:')
print(angles_all.mean(axis=0))

print('\nMean principal angle (overall):')
print(angles_all.mean())


# 3D projection

In [ ]:
Z = proj[:, :3]   # shape (T, 3)

# Plot trajectory with time ordering
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa

fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection='3d')

ax.plot(
    Z[:, 0],
    Z[:, 1],
    Z[:, 2],
    linewidth=0.8,
    alpha=0.8
)

ax.set_xlabel('Subspace dim 1')
ax.set_ylabel('Subspace dim 2')
ax.set_zlabel('Subspace dim 3')
ax.set_title('Neural population trajectory (3D subspace)')

plt.show()



## peri-stop 3D trajectories

In [ ]:
stop_label_df.columns

In [ ]:
Z_aligned = aligned_proj[:, :, :3]
y = stop_label_df['will_retry'].to_numpy().astype(bool)

mean_last = Z_aligned[y].mean(axis=0)     # (time, 3)
mean_not  = Z_aligned[~y].mean(axis=0)

fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection='3d')

ax.plot(
    mean_not[:, 0],
    mean_not[:, 1],
    mean_not[:, 2],
    label='Not last retry'
)

ax.plot(
    mean_last[:, 0],
    mean_last[:, 1],
    mean_last[:, 2],
    label='Last retry'
)

ax.scatter(
    mean_not[0, 0], mean_not[0, 1], mean_not[0, 2],
    marker='o', s=40
)
ax.scatter(
    mean_last[0, 0], mean_last[0, 1], mean_last[0, 2],
    marker='^', s=40
)

ax.set_xlabel('Dim 1')
ax.set_ylabel('Dim 2')
ax.set_zlabel('Dim 3')
ax.legend()
ax.set_title('Peri-stop population trajectories')

plt.show()



## Overlay trajectories

In [ ]:
from neural_data_analysis.neural_analysis_tools.neural_axes import multivariate_axis_utils

In [ ]:
reload(multivariate_axis_utils)

In [ ]:
fig, ax = multivariate_axis_utils.plot_peri_event_trajectories_3d(
    aligned_proj=aligned_proj,
    labels=stop_label_df['last_retry'].to_numpy(),
    dims=(0, 1, 2),
    n_show=30,
    colors=('tab:blue', 'tab:orange'),
    labels_text=('Not last retry', 'Last retry'),
    title='Peri-stop population trajectories'
)


## Compute distance between condition means in subspace as a function of time

In [ ]:
dist = np.linalg.norm(mean_last - mean_not, axis=1)

# Bootstrap confidence interval (this is the key)
n_boot = 1000
boot_dist = np.zeros((n_boot, len(time)))

idx_last = np.where(y)[0]
idx_not  = np.where(~y)[0]

for b in range(n_boot):
    samp_last = np.random.choice(idx_last, size=len(idx_last), replace=True)
    samp_not  = np.random.choice(idx_not,  size=len(idx_not),  replace=True)

    mean_l = Z_aligned[samp_last].mean(axis=0)
    mean_n = Z_aligned[samp_not].mean(axis=0)

    boot_dist[b] = np.linalg.norm(mean_l - mean_n, axis=1)

# Plot with error band
low = np.percentile(boot_dist, 2.5, axis=0)
high = np.percentile(boot_dist, 97.5, axis=0)

plt.figure(figsize=(6,4))
plt.fill_between(time, low, high, alpha=0.3)
plt.plot(time, dist, color='k')
plt.axvline(0, linestyle='--', color='k')
plt.ylabel('Distance between condition means')
plt.xlabel('Time from stop (s)')
plt.title('Population separation with 95% CI')
plt.show()


## Axis-wise error bars (only for axis 1)

In [ ]:
# Is the dominant latent variable significantly different between conditions over time?

axis1_last = Z_aligned[y, :, 0]
axis1_not  = Z_aligned[~y, :, 0]

mean_last = axis1_last.mean(axis=0)
sem_last  = axis1_last.std(axis=0) / np.sqrt(axis1_last.shape[0])

mean_not = axis1_not.mean(axis=0)
sem_not  = axis1_not.std(axis=0) / np.sqrt(axis1_not.shape[0])

plt.figure(figsize=(6,4))

plt.fill_between(time, mean_last-sem_last, mean_last+sem_last, alpha=0.3)
plt.plot(time, mean_last, label='Last retry')

plt.fill_between(time, mean_not-sem_not, mean_not+sem_not, alpha=0.3)
plt.plot(time, mean_not, label='Not last retry')

plt.axvline(0, linestyle='--', color='k')
plt.legend()
plt.ylabel('Axis 1 projection')
plt.xlabel('Time from stop (s)')
plt.title('Axis 1 with SEM')
plt.show()


## 2D confidence ellipses (for figures)

This is for visualization, not statistics.

What it shows is: Where does each condition live in subspace, and how variable is it?

In [ ]:
from matplotlib.patches import Ellipse
import numpy.linalg as LA

def plot_cov_ellipse(mean, cov, ax, n_std=2.0, **kwargs):
    vals, vecs = LA.eigh(cov)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]

    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
    width, height = 2 * n_std * np.sqrt(vals)

    ell = Ellipse(xy=mean, width=width, height=height,
                  angle=theta, **kwargs)
    ax.add_patch(ell)

t0 = np.argmin(np.abs(time - 0.1))  # e.g. 100 ms post-stop

X_last = Z_aligned[y, t0, :2]
X_not  = Z_aligned[~y, t0, :2]

fig, ax = plt.subplots()
ax.scatter(X_not[:,0], X_not[:,1], alpha=0.3)
ax.scatter(X_last[:,0], X_last[:,1], alpha=0.3)

plot_cov_ellipse(X_not.mean(0), np.cov(X_not.T), ax, edgecolor='blue', fill=False)
plot_cov_ellipse(X_last.mean(0), np.cov(X_last.T), ax, edgecolor='orange', fill=False)

ax.set_xlabel('Dim 1')
ax.set_ylabel('Dim 2')
ax.set_title('Population states with covariance ellipses')
plt.show()


# Planning vs execution

In [ ]:
stop!

In [ ]:
behavior_cols = [
    'pre_stop',
    'post_stop',
    'retry',
]


# Multitask linear with explicit interactions

In [ ]:
analyzer = MultivariateAxisAnalyzer(
    spikes_df=spikes_df,
    events_df=stop_df,
    behavior_df=trial_df,
    event_col='stop_time',
    behavior_cols=['retry', 'same_side', 'captured'],
)

axis_info = analyzer.compute_axes(
    window_ms=(-200, 300),
    interaction_pairs=[(0, 1), (0, 2)],
    fit_backend='linear',
    linear_method='mt_elasticnet',
    alpha=1e-2,
    l1_ratio=0.3,
)

W = axis_info['axes']
proj = axis_info['projection']  # (T, n_targets_aug)


# Reduced-rank multitask (shared subspace)

In [ ]:
axis_info = analyzer.compute_axes(
    window_ms=(-200, 300),
    interaction_pairs=[(0, 1)],
    fit_backend='linear',
    rank=3,
    alpha=1.0,
)

proj = axis_info['projection']  # (T, 3)


# Nonlinear axes (MLP multi-output)

In [ ]:
axis_info = analyzer.compute_axes(
    window_ms=(-200, 300),
    interaction_pairs=[(0, 1)],
    fit_backend='nonlinear',
    nonlinear_hidden=(64, 32),
    nonlinear_alpha=1e-4,
)

proj = axis_info['projection']  # (T, n_targets_aug)
project_fn = axis_info['project_fn']


# CEBRA

In [ ]:
analyzer.behavior_cols

In [ ]:
reload(cebra_analyzer)

In [ ]:
from neural_data_analysis.neural_analysis_tools.neural_axes import cebra_analyzer

analyzer = cebra_analyzer.CEBRAAnalyzer(
    spikes_df=pn.spikes_df,
    behavior_df=stop_label_df,
    event_col='time',
    behavior_cols=columns,
    bin_width_ms=10,
)

embedding, model = analyzer.fit_cebra(
    output_dim=3,
    conditional='behavior',
    max_iterations=8000,
)

aligned_embed, time = analyzer.extract_peri_event_embeddings(
    window_ms=(-300, 500)
)

results = analyzer.permutation_test_peri_event_distance(
    aligned_embed=aligned_embed,
    labels=stop_label_df['last_retry'].to_numpy(),
    n_perm=1000,
)

obs_dist = results['obs_dist']
perm_dist = results['perm_dist']
p_vals = results['p_values']
p_global = results['p_global']

low = np.percentile(perm_dist, 2.5, axis=0)
high = np.percentile(perm_dist, 97.5, axis=0)

plt.figure(figsize=(6,4))
plt.fill_between(time, low, high, alpha=0.3, label='Permutation null (95%)')
plt.plot(time, obs_dist, color='k', label='Observed')
plt.axvline(0, linestyle='--', color='k')
plt.xlabel('Time from stop (s)')
plt.ylabel('Distance in CEBRA space')
plt.legend()
plt.title(f'Permutation test (global p = {p_global:.3g})')
plt.show()


In [ ]:
import cebra
print(cebra.__version__)


In [ ]:
!pip install cebra